In [ ]:
Main Code with unique column:

In [ ]:
import jaydebeapi
import psycopg2
import sys
import time
import threading

def connect_progress(jdbc_driver_jar, jdbc_driver_class, jdbc_url, db_username, db_password):
    try:
        conn = jaydebeapi.connect(
            jdbc_driver_class,
            jdbc_url,
            [db_username, db_password],
            jdbc_driver_jar
        )
        return conn
    except Exception as e:
        print(f"Error connecting to Progress database: {e}")
        sys.exit(1)

def connect_postgresql(host, port, database, username, password):
    try:
        conn = psycopg2.connect(
            host=host,
            port=port,
            database=database,
            user=username,
            password=password
        )
        return conn
    except Exception as e:
        print(f"Error connecting to PostgreSQL database: {e}")
        sys.exit(1)

def map_data_types(progress_type):
    type_mapping = {
        'CHAR': 'CHAR',
        'CHARACTER': 'CHAR',
        'VARCHAR': 'VARCHAR',
        'INTEGER': 'INTEGER',
        'INT': 'INTEGER',
        'SMALLINT': 'SMALLINT',
        'INT64': 'BIGINT',
        'DECIMAL': 'NUMERIC',
        'NUMERIC': 'NUMERIC',
        'FLOAT': 'DOUBLE PRECISION',
        'DOUBLE': 'DOUBLE PRECISION',
        'DATE': 'DATE',
        'DATETIME': 'TIMESTAMP',
        'TIMESTAMP': 'TIMESTAMP',
        'LOGICAL': 'BOOLEAN',
    }
    return type_mapping.get(progress_type.upper(), 'TEXT')

def get_table_schema(progress_conn, table_name):
    try:
        metadata = progress_conn.jconn.getMetaData()
        result_set = metadata.getColumns(None, 'pub', table_name.upper(), None)
        schema = []
        while result_set.next():
            column_name = result_set.getString("COLUMN_NAME")
            data_type = result_set.getString("TYPE_NAME")
            precision = result_set.getInt("COLUMN_SIZE")
            scale = result_set.getInt("DECIMAL_DIGITS")
            nullable = result_set.getInt("NULLABLE")
            schema.append((column_name, data_type, precision, scale, nullable))
        return schema
    except Exception as e:
        print(f"Error fetching schema for table {table_name}: {e}")
        return []

def create_table_postgresql(pg_conn, source_table_name, schema, target_schema='qad_brazil'):
    target_table_name = f"{target_schema}.{source_table_name}_mona"
    cursor = pg_conn.cursor()
    columns = []
    for column in schema:
        column_name, data_type, precision, scale, nullable = column
        pg_data_type = map_data_types(data_type)
        if pg_data_type == 'NUMERIC' and precision > 0:
            pg_data_type = f"{pg_data_type}({precision}, {scale})"
        elif pg_data_type in ('CHAR', 'VARCHAR') and precision > 0:
            pg_data_type = f"{pg_data_type}({precision})"
        nullability = "NOT NULL" if nullable == 0 else ""
        columns.append(f"{column_name} {pg_data_type} {nullability}")
    columns_str = ", ".join(columns)
    create_table_query = f"CREATE TABLE IF NOT EXISTS {target_table_name} ({columns_str});"
    cursor.execute(create_table_query)
    pg_conn.commit()
    cursor.close()
    return target_table_name

def handle_null_values(row, column_names, not_null_columns):
    return [
        value if value is not None else
        (get_default_value(column_names[i]) if column_names[i] in not_null_columns else None)
        for i, value in enumerate(row)
    ]

def migrate_data(progress_conn, pg_conn, table_name):
    print(f"Starting migration for table {table_name}...")
    schema = get_table_schema(progress_conn, table_name)
    if not schema:
        print(f"No schema found for table {table_name}. Skipping.")
        return
    target_table_name = create_table_postgresql(pg_conn, table_name, schema, 'qad_brazil')
    pg_cursor = pg_conn.cursor()
    pg_cursor.execute(f"SELECT COUNT(*) FROM {target_table_name};")
    rows_already_inserted = pg_cursor.fetchone()[0]
    print(f"Rows already in target table: {rows_already_inserted}")
    progress_cursor = progress_conn.cursor()
    query = f"SELECT * FROM pub.{table_name}"
    print(f"Fetching rows from Progress: {query}")
    progress_cursor.execute(query)
    batch_size = 100000
    rows_skipped = 0
    total_rows_processed = 0
    columns = [col[0] for col in schema]
    not_null_columns = [col[0] for col in schema if col[4] == 0]
    while True:
        rows = progress_cursor.fetchmany(batch_size)
        if not rows:
            break
        total_rows_processed += len(rows)
        if rows_skipped + len(rows) <= rows_already_inserted:
            rows_skipped += len(rows)
            print(f"Skipped {len(rows)} rows, total skipped: {rows_skipped}")
            continue
        elif rows_skipped < rows_already_inserted:
            rows_to_skip = rows_already_inserted - rows_skipped
            rows = rows[rows_to_skip:]
            rows_skipped += rows_to_skip
            print(f"Skipped {rows_to_skip} rows, total skipped: {rows_skipped}")
        rows = [handle_null_values(row, columns, not_null_columns) for row in rows]
        print(f"Migrating {len(rows)} rows...")
        placeholders = ", ".join(["%s"] * len(schema))
        columns_str = ", ".join(columns)
        insert_query = f"INSERT INTO {target_table_name} ({columns_str}) VALUES ({placeholders})"
        try:
            pg_cursor.executemany(insert_query, rows)
            pg_conn.commit()
        except Exception as e:
            print(f"Error inserting data into PostgreSQL: {e}")
            pg_conn.rollback()
    progress_cursor.close()
    pg_cursor.close()
    print(f"Migration complete for table {table_name}. Total rows processed: {total_rows_processed}")

def keep_running():
    try:
        while True:
            time.sleep(300)
    except KeyboardInterrupt:
        print("Script terminated by user.")

def main():
    progress_conn = connect_progress(
        jdbc_driver_jar="path/to/your/openedge_driver.jar",
        jdbc_driver_class="com.ddtek.jdbc.openedge.OpenEdgeDriver",
        jdbc_url="jdbc:datadirect:openedge://your_host:your_port;databaseName=your_database",
        db_username="your_username",
        db_password="your_password"
    )
    pg_conn = connect_postgresql(
        host="your_pg_host",
        port="your_pg_port",
        database="your_pg_database",
        username="your_pg_username",
        password="your_pg_password"
    )
    tables = ["your_table_name"]
    for table_name in tables:
        migrate_data(progress_conn, pg_conn, table_name)
    threading.Thread(target=keep_running).start()

if __name__ == "__main__":
    main()


In [ ]:
Main Code without unique column by using row Hashing:

In [ ]:
import jaydebeapi
import psycopg2
import sys
import time
import io
import csv
import logging
import traceback
import os
from decimal import Decimal, InvalidOperation
import hashlib

# JVM option for Java encoding error handling
os.environ["JAVA_TOOL_OPTIONS"] = "-DallowExceptionForInvalidEncodedData=false"

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

# Constants
PROGRESS_SCHEMA = 'PUB'

def connect_progress(jdbc_driver_jar, jdbc_driver_class, jdbc_url, db_username, db_password):
    return jaydebeapi.connect(jdbc_driver_class, jdbc_url, [db_username, db_password], jdbc_driver_jar)

def connect_postgresql(host, port, database, username, password):
    return psycopg2.connect(host=host, port=port, database=database, user=username, password=password, client_encoding='UTF8')

def check_table_exists(pg_conn, schema_name, table_name):
    with pg_conn.cursor() as cursor:
        cursor.execute("""
            SELECT EXISTS (
                SELECT 1 FROM information_schema.tables 
                WHERE table_schema = %s AND table_name = %s
            );
        """, (schema_name, table_name))
        return cursor.fetchone()[0]

def map_data_types(progress_type):
    mapping = {
        'CHAR': 'CHAR',
        'CHARACTER': 'CHAR',
        'VARCHAR': 'VARCHAR',
        'NVARCHAR': 'VARCHAR',
        'NCHAR': 'CHAR',
        'INTEGER': 'INTEGER',
        'INT': 'INTEGER',
        'SMALLINT': 'SMALLINT',
        'INT64': 'BIGINT',
        'DECIMAL': 'NUMERIC',
        'NUMERIC': 'NUMERIC',
        'FLOAT': 'DOUBLE PRECISION',
        'DOUBLE': 'DOUBLE PRECISION',
        'DATE': 'DATE',
        'DATETIME': 'TIMESTAMP',
        'TIMESTAMP': 'TIMESTAMP',
        'LOGICAL': 'CHAR(1)'
    }
    return mapping.get(progress_type.upper(), 'TEXT')

def get_table_schema(progress_conn, table_name):
    metadata = progress_conn.jconn.getMetaData()
    result_set = metadata.getColumns(None, PROGRESS_SCHEMA, table_name.upper(), None)
    schema = []
    while result_set.next():
        schema.append((
            result_set.getString("COLUMN_NAME"),
            result_set.getString("TYPE_NAME"),
            result_set.getInt("COLUMN_SIZE"),
            result_set.getInt("DECIMAL_DIGITS"),
            result_set.getInt("NULLABLE")
        ))
    return schema

def create_table_postgresql(pg_conn, source_table_name, schema, target_schema='target_schema'):
    with pg_conn.cursor() as cursor:
        columns = []
        for column in schema:
            name, dtype, precision, scale, nullable = column
            pg_type = map_data_types(dtype)
            if pg_type == 'NUMERIC' and precision > 0:
                pg_type = f"{pg_type}({precision},{scale})"
            elif pg_type in ('CHAR', 'VARCHAR') and precision > 0:
                pg_type = f"{pg_type}({precision})"
            nullability = "NOT NULL" if nullable == 0 else ""
            columns.append(f'"{name}" {pg_type} {nullability}')
        columns.append('"row_hash" TEXT NOT NULL')
        create_sql = f'CREATE TABLE IF NOT EXISTS "{target_schema}"."{source_table_name}" ({", ".join(columns)});'
        cursor.execute(create_sql)
        pg_conn.commit()
        logger.info(f"Table {target_schema}.{source_table_name} created successfully.")

def get_existing_hashes(pg_conn, schema_name, table_name):
    with pg_conn.cursor() as cursor:
        cursor.execute(f'SELECT row_hash FROM "{schema_name}"."{table_name}"')
        return set(row[0] for row in cursor.fetchall())

def adjust_column_size(pg_conn, schema_name, table_name, column_name, new_size):
    with pg_conn.cursor() as cursor:
        try:
            alter_query = f'ALTER TABLE "{schema_name}"."{table_name}" ALTER COLUMN "{column_name}" TYPE VARCHAR({new_size})'
            cursor.execute(alter_query)
            pg_conn.commit()
            logger.info(f"Adjusted column {column_name} to size {new_size}.")
        except Exception as e:
            logger.error(f"Failed adjusting column {column_name}: {e}")
            pg_conn.rollback()

def migrate_data_with_hash(progress_details, pg_details, table_name, schema_name='target_schema'):
    try:
        table_name_pg = table_name.lower()
        table_name_progress = table_name.upper()

        progress_conn = connect_progress(*progress_details)
        pg_conn = connect_postgresql(*pg_details)
        schema = get_table_schema(progress_conn, table_name)
        if not schema:
            logger.error("No schema found.")
            return

        if not check_table_exists(pg_conn, schema_name, table_name_pg):
            create_table_postgresql(pg_conn, table_name_pg, schema, schema_name)

        column_names = [col[0] for col in schema]
        select_str = ", ".join([
            f'SUBSTRING("{col}", 1, {prec}) AS "{col}"' if dtype.upper() in ['CHAR', 'VARCHAR', 'NVARCHAR', 'NCHAR'] and prec > 0 else f'"{col}"'
            for col, dtype, prec, _, _ in schema
        ])

        progress_cursor = progress_conn.cursor()
        progress_cursor.execute(f'SELECT {select_str} FROM {PROGRESS_SCHEMA}."{table_name_progress}"')
        columns = [desc[0] for desc in progress_cursor.description]

        pg_cursor = pg_conn.cursor()
        pg_cursor.execute(f"""
            SELECT column_name, data_type, character_maximum_length 
            FROM information_schema.columns 
            WHERE table_schema = %s AND table_name = %s
        """, (schema_name, table_name_pg))
        col_info = {row[0]: {'type': row[1], 'max_length': row[2]} for row in pg_cursor.fetchall()}

        existing_hashes = get_existing_hashes(pg_conn, schema_name, table_name_pg)
        logger.info(f"Fetched {len(existing_hashes)} existing row_hash values.")

        buffer = io.StringIO()
        writer = csv.writer(buffer)
        total = 0
        batch_size = 10

        while True:
            rows = progress_cursor.fetchmany(batch_size)
            if not rows:
                break
            for row in rows:
                values = []
                raw_values = []
                for idx, val in enumerate(row):
                    col = columns[idx]
                    info = col_info.get(col, {'type': 'text', 'max_length': None})
                    max_len = info['max_length']
                    if val is None:
                        values.append("\\N")
                        raw_values.append('')
                    else:
                        if isinstance(val, str) and max_len and len(val) > max_len:
                            adjust_column_size(pg_conn, schema_name, table_name_pg, col, len(val))
                            val = val[:max_len]
                        elif isinstance(val, Decimal):
                            val = str(val)
                        values.append(val)
                        raw_values.append(str(val))
                hash_input = '||'.join(raw_values)
                row_hash = hashlib.md5(hash_input.encode('utf-8')).hexdigest()
                if row_hash in existing_hashes:
                    continue
                values.append(row_hash)
                writer.writerow(values)
                total += 1

        buffer.seek(0)
        if total > 0:
            col_list = ', '.join([f'"{c}"' for c in columns] + ['"row_hash"'])
            copy_query = f'COPY "{schema_name}"."{table_name_pg}" ({col_list}) FROM STDIN WITH CSV NULL \'\\N\''
            pg_cursor.copy_expert(copy_query, buffer)
            pg_conn.commit()
            logger.info(f"Copied {total} new rows to {schema_name}.{table_name_pg}.")
        else:
            logger.info("No new rows to insert.")
        pg_cursor.close()
        progress_cursor.close()
        pg_conn.close()
        progress_conn.close()
    except Exception as e:
        logger.error(f"Migration error: {e}")
        logger.debug(traceback.format_exc())

if __name__ == "__main__":
    progress_details = (
        r"/path/to/openedge_driver.jar",       # JDBC driver jar path (hidden)
        'com.ddtek.jdbc.openedge.OpenEdgeDriver',
        'jdbc:datadirect:openedge://<PROGRESS_HOST>:<PORT>;databaseName=<DATABASE>;',
        '<PROGRESS_USERNAME>',
        '<PROGRESS_PASSWORD>'
    )
    pg_details = (
        '<POSTGRES_HOST>',
        '5432',
        '<POSTGRES_DATABASE>',
        '<POSTGRES_USERNAME>',
        '<POSTGRES_PASSWORD>'
    )
    table_name = 'your_table_name'  # Use your actual table name here
    migrate_data_with_hash(progress_details, pg_details, table_name, 'target_schema')
